In [33]:
import os
import csv
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

In [34]:
working_dir = os.path.expanduser("~/projects/NLP/NLP-language-detection/data/clean_data")

In [35]:
# Load train and test set from text
train_df = pd.read_csv(f"{working_dir}/train.txt", sep="\t", header=None, names=["text", "label"], quoting=csv.QUOTE_NONE)
test_df = pd.read_csv(f"{working_dir}/test.txt", sep="\t", header=None, names=["text"], quoting=csv.QUOTE_NONE)
test_labels = pd.read_csv(f"{working_dir}/test_labels.txt", header=None, names=["label"], quoting=csv.QUOTE_NONE)

In [36]:
# Visualize train data
train_df

,text,label
0,156وحدة سكنية من مختلف الصيغ عبر بلديات الولاي...,ar
1,٣- تدني مستوي خدمات ما بعد البيع وتشغيل اطفال ...,ar
2,٣- ثم الخروج بنهاية الدوام والانضمام للمواكب و...,ar
3,٤ ٪ ومستوى رضا الزبائن الـ ٨٤.,ar
4,٥- كيفية تمثيل النازحين و اللاجئين و المجتمع ...,ar
...,...,...
62995,ຊົມຄວາມງາມຂອງຕາດຕະແລໃນລະດູຝົນ,lo
62996,ອິນເດຍກໍາລັງພົບກັບບັນຫາເຊື້ອລາດໍາລະບາດ ເຊິ່ງສາ...,lo
62997,ໂຮມຊຸມນຸມ ລະນຶກວັນຄ້າຍວັນເກີດປະທານ ໂຮ່ຈີມິງ ຄົ...,lo
62998,ສານຂອງທ່ານນາຍົກລັດຖະມົນຕີ ທອງສິງ ທຳມະວົງ ເນື່ອ...,lo


In [37]:
# Visualize test data
test_df

,text
0,ومع أن أحداً من اللاعبين الآخرين تمكن طول المو...
1,ومع انسحاب الحريري من السباق الحكومي، يبقى في ...
2,ومع بدء العملية التركية في شمال سوريا، أشار ال...
3,ومع تداول وسائل إعلام عراقية وأخرى عربية لاسم ...
4,ومع تزايد الضغوط قامت الصحيفة للمرة الثالثة بت...
...,...
6995,ຫົວໜ້າກຸ່ມລີ້ໄພຄົນເບລາຣຸດ ຖືກພົບແຂວນຄໍ ໃນຢູເຄຣນ
6996,ການຄ້າໄທເຂົ້າຊີ້ແຈງຕໍ່ການຄ້າລາວຫຼັງມີປະກາດຫ້າມ...
6997,ຫຼາຍເສັ້ນທາງບໍ່ມີລາຍທາງ ໂຍທາຄວນເບິ່ງຄືນ
6998,ຈີນນຳເຮືອບິນລຳລຽງທາງທະຫານຜະລິດເອງຮຸ່ນຫລ້າສຸດອອ...


In [38]:
# Visualize test labels
test_labels

,label
0,ar
1,ar
2,ar
3,ar
4,ar
...,...
6995,lo
6996,lo
6997,lo
6998,lo


In [39]:
# Shuffle training data and visualize after shuffling
train_df = train_df.sample(frac=1, random_state=69).reset_index(drop=True)
train_df

,text,label
0,على بعد فوز قيس سعيد بالرئاسة.,ar
1,Но на создание таких работ не замахивалась!,ru
2,فقد أعلنت دار مزادات جي دبليو إس كروس، أن الفس...,ar
3,Außer der angemeldeten Auftaktkundgebung um 12...,de
4,ລາວ-ຫວຽດນາມຈະປັບປຸງກົນໄກຊຸກຍູ້ເສດຖະກິດ,lo
...,...,...
62995,The charity had on Friday warned that the boat...,en
62996,"Ausdrücklich hieß es, sie habe den Verlag im S...",de
62997,ອັນຕະລາຍຈາກການມັກຍ້ຳກິນນ້ຳກ້ອນຫຼາຍເກີນໄປ,lo
62998,مانيلا: انتقد مسؤول فلبيني، الأحد، الممثلة الأ...,ar


In [40]:
# Vectorize the text using TF-IDF
vectorizer = TfidfVectorizer(analyzer='char', ngram_range=(1,3))

X_train = vectorizer.fit_transform(train_df['text'])
X_test = vectorizer.transform(test_df['text'])

y_train = train_df['label']
y_test = test_labels['label']

In [41]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((63000, 115672), (7000, 115672), (63000,), (7000,))

In [42]:
X_train, X_test

(<Compressed Sparse Row sparse matrix of dtype 'float64'
 	with 12361955 stored elements and shape (63000, 115672)>,
 <Compressed Sparse Row sparse matrix of dtype 'float64'
 	with 1346141 stored elements and shape (7000, 115672)>)

In [43]:
vectorizer.get_feature_names_out()

array([' ', ' !', ' ! ', ..., '🇿', '🇿🇦', '🇿🇦 '],
      shape=(115672,), dtype=object)

In [44]:
# Train the model with SVM
svm_model = SVC(kernel='linear', C=1)
svm_model.fit(X_train, y_train)

,C,1
,kernel,'linear'
,degree,3
,gamma,'scale'
,coef0,0.0
,shrinking,True
,probability,False
,tol,0.001
,cache_size,200
,class_weight,None
,verbose,False


In [45]:
# Making predictions
y_pred = svm_model.predict(X_test)

In [46]:
# Evaluating model
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)
print('Classification Report:')
print(classification_report(y_test, y_pred))
print('Confusion Matrix:')
print(confusion_matrix(y_test, y_pred))

Accuracy: 0.9992857142857143
Classification Report:
              precision    recall  f1-score   support

          ar       1.00      1.00      1.00      1000
          de       1.00      1.00      1.00      1000
          en       1.00      1.00      1.00      1000
          fr       1.00      1.00      1.00      1000
          lo       1.00      1.00      1.00      1000
          ru       1.00      1.00      1.00      1000
          vi       1.00      1.00      1.00      1000

    accuracy                           1.00      7000
   macro avg       1.00      1.00      1.00      7000
weighted avg       1.00      1.00      1.00      7000

Confusion Matrix:
[[1000    0    0    0    0    0    0]
 [   0  996    3    1    0    0    0]
 [   0    0 1000    0    0    0    0]
 [   0    1    0  999    0    0    0]
 [   0    0    0    0 1000    0    0]
 [   0    0    0    0    0 1000    0]
 [   0    0    0    0    0    0 1000]]


In [ ]:
# Test some sentences
sentences = [
    "Bonjour, comment ça va ?",        
    "Xin chào, bạn khỏe không?",       
    "Hello, how are you?",             
    "Привет, как дела?",               
    "Guten Tag, wie geht es Ihnen?",   
    "مرحبا، كيف حالك؟",                 
    "ສະບາຍດີ, ເຈົ້າສະບາຍດີບໍ?" # Lao
]

# Transform sentences to TF-IDF vectors
X_new = vectorizer.transform(sentences)

# Predict labels
predictions = svm_model.predict(X_new)

# Show results with predicted labels
for sent, pred in zip(sentences, predictions):
    print(f"Input: {sent}\nPredicted language: {pred}\n")

Input: Bonjour, comment ça va ?
Predicted language: fr

Input: Xin chào, bạn khỏe không?
Predicted language: vi

Input: Hello, how are you?
Predicted language: en

Input: Привет, как дела?
Predicted language: ru

Input: Guten Tag, wie geht es Ihnen?
Predicted language: de

Input: مرحبا، كيف حالك؟
Predicted language: ar

Input: ສະບາຍດີ, ເຈົ້າສະບາຍດີບໍ?
Predicted language: lo

